In [1]:
# FORECAST TOTAL
# USE OTHER MODELS (EXPONENTIAL, SMOOTHING)
# RMSE
# FORECAST BASED ON CATEGORIES
# EXCLUDE THE ONES WITH LESS THAN 80% of highest
# Separate graphs for each item
# Put data in spreadsheet (RMSE, item1, item2 etc.)
# Divide data into testing and training sets.

# DO ONE MORE MODEL
# Use new model (Exponential smoothing) (Prophet Package)
# Save data --> RMSE separate file, Item, Model, RMSE, Forecast_model (separate file)

## Looping through all categories

In [1]:
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
%matplotlib inline
df = pd.read_csv("Sales_2015-2019.csv")
df = df.drop(['Item Code', 'Item Desc', 'Quantity'], axis=1)
unique_items_list = df['Category'].unique().tolist()
print(len(unique_items_list))
data_frames_dict = dict(tuple(df.groupby('Category')))
for item in unique_items_list:
    print(item, len(data_frames_dict[item]), sep="-->")

    
    
    
RMSE_non_dynamic = []
RMSE_dynamic = []
    
for item in unique_items_list:
    print(item)
    bx1 = data_frames_dict[item]
    bx1.head(5)
    bx1['Datetime'] = pd.to_datetime(bx1['Date'])
    bx1 = bx1.set_index('Datetime')
    months = ['2015-01', '2015-02', '2015-03', '2015-04', '2015-05', '2015-06', '2015-07', '2015-08', '2015-09', '2015-10', 
             '2015-11', '2015-12',
             '2016-01', '2016-02', '2016-03', '2016-04', '2016-05', '2016-06', '2016-07', '2016-08', '2016-09', '2016-10', 
             '2016-11', '2016-12',
             '2017-01', '2017-02', '2017-03', '2017-04', '2017-05', '2017-06', '2017-07', '2017-08', '2017-09', '2017-10', 
             '2017-11', '2017-12', 
             '2018-01', '2018-02', '2018-03', '2018-04', '2018-05', '2018-06', '2018-07', '2018-08', '2018-09', '2018-10', 
             '2018-11', '2018-12', 
             '2019-01', '2019-02', '2019-03', '2019-04']
    sums=[]
    for month in months:
        sums.append(bx1[month].sum()['Total'])
    print(sums)
    bx1.drop_duplicates(subset='Date', keep='first', inplace=True)
    bx1['New Total'] = sums
    bx1 = bx1.drop(['Date', 'Category', 'Total'], axis=1)
    bx1.plot(figsize=(15, 6))
    plt.title(item)
    plt.show()
    from pylab import rcParams
    rcParams['figure.figsize'] = 11, 9

    decomposition = sm.tsa.seasonal_decompose(bx1, model='additive')
    fig = decomposition.plot()
    plt.show()

    import warnings
    import itertools
    import pandas as pd
    import numpy as np
    import statsmodels.api as sm
    import matplotlib.pyplot as plt
    plt.style.use('fivethirtyeight')

    # Define the p, d and q parameters to take any value between 0 and 2
    p = d = q = range(0, 2)

    # Generate all different combinations of p, q and q triplets
    pdq = list(itertools.product(p, d, q))

    # Generate all different combinations of seasonal p, q and q triplets
    seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]

    print('Examples of parameter combinations for Seasonal ARIMA...')
    print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[1]))
    print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[2]))
    print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[3]))
    print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[4]))

    warnings.filterwarnings("ignore") # specify to ignore warning messages


    lowest_AIC = 1000000
    for param in pdq:
        for param_seasonal in seasonal_pdq:
            try:
                mod = sm.tsa.statespace.SARIMAX(bx1,
                                                order=param,
                                                seasonal_order=param_seasonal,
                                                enforce_stationarity=False,
                                                enforce_invertibility=False)

                results = mod.fit()
                if results.aic < lowest_AIC:
                    lowest_AIC = results.aic
                    lowest_params_normal = param
                    lowest_params_seasonal = param_seasonal
                print('ARIMA{}x{}12 - AIC:{}'.format(param, param_seasonal, results.aic))
            except:
                continue

    print(lowest_AIC)
    print(lowest_params_normal)
    print(lowest_params_seasonal)

    mod = sm.tsa.statespace.SARIMAX(bx1,
                                    order=lowest_params_normal,
                                    seasonal_order=lowest_params_seasonal,
                                    enforce_stationarity=False,
                                    enforce_invertibility=False)

    results = mod.fit()

    print(results.summary().tables[1])

    pred = results.get_prediction(start=pd.to_datetime('2018-05-01'), dynamic=False)
    pred_ci = pred.conf_int()

    ax = bx1.plot(label='observed')
    pred.predicted_mean.plot(ax=ax, label='One-step ahead Forecast', alpha=.7)

    ax.fill_between(pred_ci.index,
                    pred_ci.iloc[:, 0],
                    pred_ci.iloc[:, 1], color='k', alpha=.2)

    ax.set_xlabel('Date')
    ax.set_ylabel('Sales')
    plt.legend()

    plt.show()

    y_forecasted = pred.predicted_mean
    y_truth = bx1['2018-05-01':]['New Total']

    # Compute the mean square error
    mse = ((y_forecasted - y_truth) ** 2).mean()**0.5
    RMSE_non_dynamic.append(mse)
    print('The Mean Squared Error of our forecasts is {}'.format(round(mse, 2)))
    # print(y_forecasted)
    # print(y_truth)

    pred_dynamic = results.get_prediction(start=pd.to_datetime('2018-05-01'), dynamic=True, full_results=True)
    pred_dynamic_ci = pred_dynamic.conf_int()

    ax = bx1.plot(label='observed', figsize=(20, 15))
    pred_dynamic.predicted_mean.plot(label='Dynamic Forecast', ax=ax)

    ax.fill_between(pred_dynamic_ci.index,
                    pred_dynamic_ci.iloc[:, 0],
                    pred_dynamic_ci.iloc[:, 1], color='k', alpha=.25)

    ax.fill_betweenx(ax.get_ylim(), pd.to_datetime('2018-05-01'), bx1.index[-1],
                     alpha=.1, zorder=-1)

    ax.set_xlabel('Date')
    ax.set_ylabel('Sales')

    plt.legend()
    plt.show()

    # Extract the predicted and true values of our time series
    y_forecasted = pred_dynamic.predicted_mean
    y_truth = bx1['2018-05-01':]['New Total']

    # Compute the mean square error
    mse = ((y_forecasted - y_truth) ** 2).mean()**0.5
    RMSE_dynamic.append(mse)

    print('The Mean Squared Error of our forecasts is {}'.format(round(mse, 2)))


    # Get forecast 500 steps ahead in future
    pred_uc = results.get_forecast(steps=50)

    # Get confidence intervals of forecasts
    pred_ci = pred_uc.conf_int()


    ax = bx1.plot(label='observed', figsize=(20, 15))
    pred_uc.predicted_mean.plot(ax=ax, label='Forecast')
    ax.fill_between(pred_ci.index,
                    pred_ci.iloc[:, 0],
                    pred_ci.iloc[:, 1], color='k', alpha=.25)
    ax.set_xlabel('Date')
    ax.set_ylabel('Sales')

    plt.legend()
    plt.show()

print(RMSE_non_dynamic)
print(RMSE_dynamic)

ImportError: cannot import name 'factorial' from 'scipy.misc' (/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/scipy/misc/__init__.py)

In [3]:
print(RMSE_non_dynamic)
print(RMSE_dynamic)
print(min(RMSE_non_dynamic))
print(min(RMSE_dynamic))
for item in unique_items_list:
    print(item, len(data_frames_dict[item]), sep="-->")

[4085.5007682154674, 1654.4580597508473, 2862.832331915578, 3614.0467127109314, 564.3489455788434, 2318.1245369642684, 1742.8292048361054, 7467.212012996072, 4302.717187067364]
[4172.090652781254, 3944.9728004325266, 7101.735631822121, 6575.513887132069, 544.2516793613173, 3588.0796389705465, 4364.34130564269, 7769.973871626993, 4711.60434130365]
564.3489455788434
544.2516793613173
Fries-->415
Sausages-->288
Entries-->285
Apetizers-->413
Desserts-->102
Sandwichs-->494
Pizzas-->1066
Tapas-->362
Hamburgers-->780
